## Defining the Problem Statement
We are analyzing job postings in Saudi Arabia to understand key employment trends. This includes insights on job distribution by region, gender preferences, and salary expectations.

In [1]:
! pip install ydata_profiling
! pip install ipyvizzu

In [4]:
! pip install --upgrade numba

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\manar\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\manar\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\manar\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\manar\anaconda3\lib\http\client.py", line 462, in read
    n = self.readinto(b)
  File "C:\Users\manar\anaconda3\lib\http\client.py", line 506, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\manar\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\manar\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\manar\anaconda3\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
soc

In [ ]:
! pip uninstall numba -y
! pip install numba

##  Loading the data

The dataset consists of job postings collected from Jadarat.

In [11]:
! pip install matplotlib==3.6.0


  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.9.4
    Uninstalling matplotlib-3.9.4:
      Successfully uninstalled matplotlib-3.9.4


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\manar\\anaconda3\\Lib\\site-packages\\~atplotlib\\ft2font.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import ipyvizzu as ipy
from IPython.display import display
import matplotlib.pyplot as plt

## This statement allows the visuals to render within your Jupyter Notebook.
%matplotlib inline


SystemError: initialization of _internal failed without raising an exception

In [ ]:
# Read in the csv file and convert to a Pandas dataframe
data = pd.read_csv('Jadarat_data.csv')

### Viewing the dataframe


In [16]:
data.head(20)  # Preview the first few rows

,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
0,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,شركة مقر العالم للسفريات,1-317262,خاص,متوسطة فئة أ,أنشطة وكالات السياحة والسفر,"['Language data', 'اللغة الانجليزية', 'محترف']",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350419,0 Years,both
1,بائع,27/05/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركة عالم الكهرباء للمقاولات,4-1324428,خاص,متوسطة فئة ب,تركيب انظمة التبريد وتكييف الهواء وصيانتها واص...,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary', '5000.0']",دوام كامل,0 / 3,20202026350389,0 Years,both
2,أخصائي عمليات موارد بشرية,27/05/1444,['تنفيذ الإجراءات والأنظمة والنماذج الخاصة بمر...,[' تنفيذ الإجراءات والأنظمة والنماذج الخاصة ...,شركه دار السلام,1-155294,خاص,متوسطة فئة أ,ترميمات المباني السكنية والغير سكنية,"['Language data', 'الانجليزيه', 'محترف']",الرياض,ATH THUMA...,"['Salary', '4000.0']",دوام كامل,0 / 2,20202026350347,2 Years,both
3,ميكانيكي سيارات,27/05/1444,['تشخيص أعطال السيارات وإصلاحها وتنفيذ برامج ا...,[' فحص أداء المعدّات الكهربائية والميكانيكية...,مؤسسة لمكو لغيار الزيوت,8-1925495,خاص,صغيرة فئة ب,NaN,"['Skill data', 'صيانة السيارات وتقييم الاعطال'...",المنطقة الشرقية,AL HUFUF...,"['Salary', '5000.0']",دوام كامل,0 / 10,20202026350219,0 Years,M
4,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,مؤسسة فكرة اليمامة للمقاولات,1-2356639,خاص,كبيرة,الإنشاءات العامة للمباني السكنية,"['Skill data', 'تحمل ضغط العمل', 'محترف', 'Lan...",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350043,0 Years,both
5,مراقب كاميرات أمنية,27/05/1444,['الإشراف على عمليات كاميرات المراقبة الأمنية،...,[' الإشراف على عمليات كاميرات المراقبة، والت...,مطاعم تراث الدمشقية للوجبات السريعة,1-1527188,خاص,صغيرة فئة ب,تشغيل المقاصف والكفتيريات على أساس إمتياز محدد...,"['Skill data', 'اجادة الحاسب والتقنية ', 'متوس...",الرياض,AR RIYADH...,"['Salary', '4500.0']",عمل عن بعد,0 / 1,20202026350031,0 Years,both
6,سكرتير تنفيذي,27/05/1444,['تنظيم المواعيد وإعداد المذكرات وإجراء المراس...,[' تنظيم مواعيد المدير/الرئيس وتذكيره بها، و...,شركة بيانات و دعم القرار للاستشارات في مجال ال...,9-2027654,خاص,صغيرة فئة أ,إعادة تأهيل وهيكلة العمليات الإدارية والمالية ...,"['Language data', 'لغة انجليزية', 'محترف']",الرياض,AR RIYADH...,"['Salary', '6000.0']",دوام كامل,0 / 2,20202026349999,4 Years,F
7,مدير مكتب,27/05/1444,['الإشراف المباشرعلى أنشطة إدارة وتخطيط وتنظيم...,[' إدارة وتوجيه عمليات وإجراءات تنسيق اجتماع...,شركة بيانات و دعم القرار للاستشارات في مجال ال...,9-2027654,خاص,صغيرة فئة أ,إعادة تأهيل وهيكلة العمليات الإدارية والمالية ...,"['Language data', 'لغة انجليزية', 'محترف']",الرياض,AR RIYADH...,"['Salary', '8000.0']",دوام كامل,0 / 1,20202026349997,4 Years,F
8,أخصائي إدارة مشاريع,27/05/1444,['تحليل عناصر وفعاليات المشروع ووضع الخطط والب...,[' تحليل عناصر وفعاليات المشروع وارتباطها وت...,شركة بيانات و دعم القرار للاستشارات في مجال ال...,9-2027654,خاص,صغيرة فئة أ,إعادة تأهيل وهيكلة العمليات الإدارية والمالية ...,"['Language data', 'لغة انجليزية', 'محترف', 'Pr...",الرياض,AR RIYADH...,"['Salary', '10000.0']",دوام كامل,0 / 3,20202026349992,2 Years,F
9,مصمم جرافيك,26/05/1444,['تطوير الأفكار الإبداعية والمفاهيم لتوصيل الر...,[' التنسيق مع مالك التصميم (عملاء محليون أو ...,شركة بيانات و دعم القرار للاستشارات في مجال ال...,9-2027654,خاص,صغيرة فئة أ,إعادة تأهيل وهيكلة العمليات الإدارية والمالية ...,"['Language data', 'لغة انجليزية', 'متوسط']",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 2,20202026349583,4 Years,F


In [ ]:
data.tail(20)  # Preview the last few rows

,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
1450,مهندس معماري,25/06/1444,['تصميم المباني الجديدة أو الملحقات أو التعديل...,[' وضع التصاميم لمشاريع الإنشاءات وإدارة فري...,مكتب هندسي عبدالمحسن سالم موسى سرحان,11-183776,خاص,صغيرة فئة ب,أنشطة الاستشارات الهندسية والمعمارية,NaN,الرياض,ABA ASLAB...,"['Salary', '7000.0', 'Benefits', 'بدون']",دوام كامل,0 / 1,20202026404473,2 Years,M
1451,فني مختبر طبي,25/06/1444,['تفقد وتشغيل وخدمة معدات المختبر،أخذ عينات ال...,[' تفقد وتشغيل وخدمة معدات حفظ وفحص العينات ...,مجمع ابتسامة العائلة الطبي,1-2599562,خاص,NaN,NaN,"['Professional certificate \\ License data', '...",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 2,20202026404173,0 Years,both
1452,بائع,25/06/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركة فيض الغذاء المحدودة شركة شخص واحد,9-1968507,خاص,صغيرة فئة ب,البيع بالتجزئة للشوكولاتة و الكاكاو,"['Skill data', 'استخدام الحاسب الآلي', 'متوسط'...",مكة المكرمة,JEDDAH...,"['Salary', '4200.0', 'Benefits', 'تأمين طبي حس...",دوام كامل,0 / 6,20202026404057,2 Years,M
1453,مساعد إداري,25/06/1444,['متابعة تنفيذ القرارات الادارية الصادرة عن ال...,[' تنفيذ التعليمات الادارية الصادرة عن الرئي...,شركة روافد الصحة العالمية(شركة شخص واحد),1-225115,خاص,متوسطة فئة أ,البيع بالتجزئة للأجهزة والمعدات والمستلزمات ال...,"['Language data', 'English ', 'محترف']",الرياض,AR RIYADH...,"['Salary', '5000.0', 'Benefits', 'بدل المواصلا...",دوام كامل,0 / 1,20202026403991,2 Years,F
1454,أخصائي عمليات موارد بشرية,25/06/1444,['تنفيذ الإجراءات والأنظمة والنماذج الخاصة بمر...,[' تنفيذ الإجراءات والأنظمة والنماذج الخاصة ...,شركـة ذوق للتجارة,1-1526321,خاص,صغيرة فئة ب,البيع بالتجزئة للوكلاء بالعمولة بدون محلات,NaN,الرياض,AR RIYADH...,"['Salary', '5556.0']",دوام كامل,0 / 1,20202026403956,0 Years,F
1455,فني تصميم داخلي,24/06/1444,['تصميم أنظمة الألوان وطرز الأثاث، والمواد الم...,[' وضع التصاميم للمشروعات الداخلية، وإدارة ف...,شركة نمو للبحوث والدراسات (شركة شخص واحد),1-2166711,خاص,صغيرة فئة أ,مراكز البحوث والدراسات الاجتماعية,NaN,الرياض,AR RIYADH...,"['Salary', '5000.0', 'Benefits', 'عموله']",دوام كامل,0 / 1,20202026403328,2 Years,both
1456,محامي,24/06/1444,['دراسة القضية وتجهيز ملفها وجمع البينات الخطي...,[' استقبال القضية وتحديد نوعها وتحليل عناصره...,شركة فيصل عبد الله الفوزان وشركاؤه للمحاماة,1-2664602,خاص,NaN,أنشطة المحاماة والاستشارات القانونية,"['Skill data', 'التعامل مع برامج الأوفيس', 'مح...",الرياض,AR RIYADH...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026403115,0 Years,both
1457,محاسب,23/06/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,شركة بوركت للتجارة,1-2290505,خاص,صغيرة فئة ب,البيع بالتجزئة لمنتجات المخابز والحلويات السكرية,"['Skill data', 'دورات محاسبية', 'محترف', 'Prof...",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,1 / 2,20202026402637,4 Years,both
1458,أخصائي خدمة عملاء,23/06/1444,['وضع وتحديد مؤشرات ومعايير الأداء والجودة في ...,[' إجراء الدراسات والتحاليل المتخصصة بهدف وض...,شركة بوركت للتجارة,1-2290505,خاص,صغيرة فئة ب,البيع بالتجزئة لمنتجات المخابز والحلويات السكرية,"['Skill data', 'مهارات الإقناع', 'محترف', 'الل...",الرياض,AR RIYADH...,"['Salary', '4000.0']",دوام كامل,1 / 5,20202026402465,4 Years,both
1459,موظف صندوق محاسبة,23/06/1444,['عد المبالغ المالية المتوافرة في آلة تسجيل ال...,[' عدّ المبالغ المالية وتسجيلها في آلة تسجيل...,شركة عنان للخدمات الغذائية,23-1946813,خاص,صغيرة فئة ب,المطاعم مع الخدمة,"['Skill data', 'استخدام الحاسب الالي', 'متوسط'...",المنطقة الشرقية,AL KHAFJI...,"['Salary', '4000.0', 'Benefits', 'بونص وبدلات']",دوام كامل,0 / 1,20202026402325,0 Years,both


In [18]:
data.shape

(1470, 18)

### Data Profiling


In [22]:
# Disblay all columns
data.columns

Index(['job_title', 'job_date', 'job_desc', 'job_tasks', 'comp_name',
       'comp_no', 'comp_type', 'comp_size', 'eco_activity', 'qualif', 'region',
       'city', 'benefits', 'contract', 'positions', 'job_post_id', 'exper',
       'gender'],
      dtype='object')

In [23]:
# Disblay all information available
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_title     1470 non-null   object
 1   job_date      1470 non-null   object
 2   job_desc      1470 non-null   object
 3   job_tasks     1470 non-null   object
 4   comp_name     1470 non-null   object
 5   comp_no       1470 non-null   object
 6   comp_type     1470 non-null   object
 7   comp_size     1456 non-null   object
 8   eco_activity  1380 non-null   object
 9   qualif        780 non-null    object
 10  region        1470 non-null   object
 11  city          1470 non-null   object
 12  benefits      1470 non-null   object
 13  contract      1470 non-null   object
 14  positions     1470 non-null   object
 15  job_post_id   1470 non-null   int64 
 16  exper         1470 non-null   object
 17  gender        1470 non-null   object
dtypes: int64(1), object(17)
memory usage: 206.8+ KB


In [25]:
# Display summary statistics for categorical columns
data.describe(include="object")

,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,exper,gender
count,1470,1470,1470,1470,1470,1470,1470,1456,1380,780,1470,1470,1470,1470,1470,1470,1470
unique,324,92,438,444,886,894,2,7,361,615,14,114,393,2,25,6,3
top,بائع,Publish date,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركه العرض المتقن للخدمات التجارية شركة مساهمة...,1-169123,خاص,صغيرة فئة ب,المطاعم مع الخدمة,"['Language data', 'الانجليزية', 'متوسط']",الرياض,AR RIYADH...,"['Salary', '4000.0']",دوام كامل,0 / 1,0 Years,both
freq,98,47,93,90,16,16,1465,635,65,28,622,523,449,1448,853,833,587


Unique values are: 2 comp_type - 7 comp_size - 2 contract - 6 exper - 3 gender

In [ ]:
data['job_title'].value_counts()

In [ ]:
data['job_date'].value_counts()

In [ ]:
data['job_desc'].value_counts()

In [ ]:
data['job_tasks'].value_counts()

In [ ]:
data['comp_name'].value_counts()

In [ ]:
data['comp_no'].value_counts()

In [ ]:
data['comp_type'].value_counts()

In [ ]:
data['comp_size'].value_counts()

In [ ]:
data['eco_activity'].value_counts()

In [ ]:
data['qualif'].value_counts()

In [ ]:
data['region'].value_counts()

In [ ]:
data['city'].value_counts()

In [ ]:
data['benefits'].value_counts()

In [ ]:
data['contract'].value_counts()

In [ ]:
data['positions'].value_counts()

In [ ]:
data['exper'].value_counts()

In [ ]:
data['gender'].value_counts()

In [24]:
# Display summary statistics for numerical columns
data.describe(include="number")

,job_post_id
count,1.470000e+03
mean,2.020203e+13
std,5.863941e+04
min,2.020203e+13
25%,2.020203e+13
50%,2.020203e+13
75%,2.020203e+13
max,2.020203e+13


In [ ]:
data['job_post_id'].value_counts()

In [ ]:
data.dtypes  # للتحقق من نوع البيانات

In [ ]:
data.unique()  # لرؤية القيم الفريدة

## Data Quality Checking and Remediation
We will check for missing values, data types, and inconsistencies.

In [ ]:
# Checking for missing values
data.isnull().sum()

In [ ]:
# Checking data types and basic info
data.info()

## Exploratory Data Analysis (EDA)
We'll use automated profiling with `ydata_profiling` to generate a summary report.

In [ ]:
# Generate data profiling report
profile = ProfileReport(data, explorative=True)
profile.to_notebook_iframe()

#### 1. Reliability:

#### 2. Timeliness:

#### 3. Consistency:

#### 4. Relevance:

#### 5. Uniqueness:

#### 6. Completeness:

#### 7. Check Accuracy:

## Data Cleaning